In [ ]:
# Cell 1: 导入必要的包
import torch
from torch.utils.data import DataLoader, Dataset
import pickle
import numpy as np
from ppq.api import espdl_quantize_onnx
from ppq.core import TargetPlatform

In [ ]:
# Cell 2: 准备校准数据集
class CalibrationDataset(Dataset):
    def __init__(self, X):
        self.X = torch.FloatTensor(X).unsqueeze(1)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx]

# 加载校准数据
with open('cal.pkl', 'rb') as f:
    X_cal, _ = pickle.load(f)

cal_dataset = CalibrationDataset(X_cal)
cal_loader = DataLoader(cal_dataset, batch_size=32, shuffle=False)

In [ ]:
# Cell 3: 执行量化
ONNX_MODEL_PATH = "gesture_model.onnx"
EXPORT_PATH = "gesture_model_int8.espdl"
input_shape = [1, 1, 96, 96]

graph = espdl_quantize_onnx(
    onnx_import_file=ONNX_MODEL_PATH,
    espdl_export_file=EXPORT_PATH,
    calib_dataloader=cal_loader,
    calib_steps=50,
    input_shape=[input_shape],
    target="esp32s3",
    num_of_bits=8,
    device="cpu",
    error_report=True,
    verbose=1
)

In [ ]:
# Cell 4: 评估量化后的模型
from ppq.executor import TorchExecutor
import time
from tqdm import tqdm

def evaluate_quantized_model(graph, test_loader, y_test):
    """
    评估量化后的模型性能
    """
    executor = TorchExecutor(graph=graph, device='cpu')
    total_time = 0
    correct = 0
    total = 0
    
    print("\n开始评估量化模型性能...")
    with torch.no_grad():
        for batch in tqdm(test_loader):
            start = time.time()
            outputs = executor.forward(inputs=batch)
            total_time += (time.time() - start)
            
            # 计算准确率
            _, predicted = torch.max(outputs[0], 1)
            total += batch.size(0)
            correct += (predicted == y_test[total-batch.size(0):total]).sum().item()

    # 计算并打印结果
    avg_time = (total_time / len(test_loader)) * 1000  # 转换为毫秒
    accuracy = (correct / total) * 100

    print(f"\n评估结果:")
    print(f"平均推理时间: {avg_time:.2f} ms")
    print(f"模型准确率: {accuracy:.2f}%")
    
    return avg_time, accuracy

# 加载测试数据
with open('test.pkl', 'rb') as f:
    X_test, y_test = pickle.load(f)

# 准备测试数据集
test_dataset = CalibrationDataset(X_test)  # 复用之前的Dataset类
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 执行评估
evaluate_quantized_model(graph, test_loader, y_test)

In [ ]:
# Cell 5: 使用量化后的模型识别一张图片
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt

def preprocess_image(image_path):
    """
    图片预处理函数
    1. 读取图片
    2. 转换为灰度图
    3. 调整大小
    4. 归一化
    """
    # 读取图片
    img = cv2.imread(image_path)
    # 转换为灰度图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 调整大小
    TARGET_SIZE = (96, 96)
    resized = cv2.resize(gray, TARGET_SIZE)
    # 归一化
    normalized = resized.astype('float32') / 255.0
    # 添加 batch 和 channel 维度
    image_tensor = torch.FloatTensor(normalized).unsqueeze(0).unsqueeze(0)
    
    return image_tensor, img

def predict_image(graph, image_tensor):
    """
    使用量化后的模型预测图像的类别置信度。
    """
    executor = TorchExecutor(graph=graph, device='cpu')
    with torch.no_grad():
        outputs = executor.forward(inputs=[image_tensor])
        confidences = torch.softmax(outputs[0], dim=1).numpy().flatten()
    
    return confidences

# 定义手势类别
gesture_labels = {
    '00': 'palm',
    '01': 'l',
    '02': 'fist',
    '03': 'thumb',
    '04': 'index',
    '05': 'ok',
    '06': 'c',
    '07': 'down'
}

# 加载并预处理图像
# 请将 "your_image.jpg" 替换为你要预测的图像的实际路径，例如："/path/to/your_image.jpg"
image_path = "frame_00_01_0009.png"  # 这里将 "your_image.jpg" 替换为你要使用的图像路径
image_tensor, original_image = preprocess_image(image_path)

# 使用量化后的模型进行预测
confidences = predict_image(graph, image_tensor)

# 打印模型输出的各个置信度，并关联类别标签
print("\n预测结果: \n")
for idx, confidence in enumerate(confidences):
    label = gesture_labels[f'{idx:02}']
    print(f"类别: {label}, 置信度: {confidence:.4f}")

# 显示原始图片
plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
plt.title("Original Image")
plt.axis('off')
plt.show()